In [8]:
import math
import csv
import cv2
import keras
import numpy as np
from keras import backend as K
from keras.optimizers import Adam
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.applications.mobilenet import preprocess_input, MobileNet
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

In [9]:
def dev_image_generator(input_feed,data_folder, bs, mode="train", aug=None):
    index = 0
    
    NUM_IMAGES = len(input_feed)
    
    while True:
        
        # initialize our batches of images and labels
        images = []
        poly_cof = []
        
        # keep looping until we reach our batch size
        while len(images) < bs:
            
            line = input_feed[index]
            index = index + 1
            if index >= len(input_feed):
                index = 0;
            
            center_img_path = line[1]
            file_name = center_img_path
#             print('./{}/{}'.format(folder, file_name))
            
            if len(file_name) > 0:
                image = cv2.imread('./{}/{}.jpg'.format(data_folder, file_name))
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image,(160,160))
                
            if image is not None:
                pixels = np.asarray(image)
                pixels = pixels.astype('float32')
                pixels = pixels/255
                images.append(pixels)
#                 images.append(np.fliplr(image))
            

                x1 = (float(line[3])/127.5) -1
                poly_cof.append([x1])
        
        # if the data augmentation object is not None, apply it
        if aug is not None:
            (images, labels) = next(aug.flow(np.array(images),
                labels, batch_size=bs))
        # yield the batch to the calling function
        yield (np.array(images), np.array(poly_cof))
    

In [10]:
## Reading all data and sepperate to train dataset and valdidate dataset ##

image_load_data = []

with open('dataset2.csv') as f:

    next(f, None)

    reader = csv.reader(f)
    for csv_line in reader:
        image_load_data.append((csv_line))

image_load_data = shuffle(image_load_data)

data_train, data_test  = train_test_split(image_load_data,test_size=0.2)


In [11]:
# initialize the number of epochs to train for and batch_size
NUM_EPOCHS = 100
BATCH_SIZE = 4

data_folder = 'dataset'

# initialize the total number of training and testing image
NUM_TRAIN_IMAGES = len(data_train)
NUM_TEST_IMAGES = len(data_test)

trainGen = dev_image_generator(data_train,data_folder,BATCH_SIZE, mode = "train", aug = None)
testGen = dev_image_generator(data_test,data_folder,BATCH_SIZE, mode = "train", aug = None)

In [12]:
# initialize our Keras model and compile it

base_model=MobileNet(input_shape=(160, 160, 3), alpha = 0.5,depth_multiplier = 1, dropout = 0.001,include_top = False, weights = "imagenet", classes = 1000, backend=keras.backend, layers=keras.layers,models=keras.models,utils=keras.utils)

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(100,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dropout(0.1)(x)
x=Dense(50,activation='relu')(x) #dense layer 3
preds=Dense(1,activation='linear')(x) #final layer with softmax activation
model=Model(inputs=base_model.input,outputs=preds)

# model.summary()

filepath="w-imp-{epoch:02d}-{val_loss:.5f}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

model.compile(optimizer='adam', loss='mse')

In [13]:
# train the network

print("[INFO] training w/ generator...")
H = model.fit_generator(trainGen, steps_per_epoch=NUM_TRAIN_IMAGES // BATCH_SIZE, 
                        validation_data=testGen, validation_steps=NUM_TEST_IMAGES // BATCH_SIZE, 
                        epochs=NUM_EPOCHS, callbacks=callbacks_list)


[INFO] training w/ generator...
Epoch 1/100
300/300 [==============================] - 24s 81ms/step - loss: 2925.2510 - val_loss: 1056.7521

Epoch 00001: val_loss improved from inf to 1056.75208, saving model to w-imp-01-1056.75208.h5
Epoch 2/100
300/300 [==============================] - 19s 63ms/step - loss: 1266.8880 - val_loss: 2618.5068

Epoch 00002: val_loss did not improve from 1056.75208
Epoch 3/100
300/300 [==============================] - 19s 64ms/step - loss: 862.0289 - val_loss: 1295.6703

Epoch 00003: val_loss did not improve from 1056.75208
Epoch 4/100
300/300 [==============================] - 19s 64ms/step - loss: 719.9590 - val_loss: 1111.9783

Epoch 00004: val_loss did not improve from 1056.75208
Epoch 5/100
300/300 [==============================] - 19s 64ms/step - loss: 588.9892 - val_loss: 1393.9788

Epoch 00005: val_loss did not improve from 1056.75208
Epoch 6/100
300/300 [==============================] - 20s 65ms/step - loss: 532.0479 - val_loss: 392.5753

Epo

KeyboardInterrupt: 

In [ ]:
model.save('road_following_model.h5')